In [4]:
sequence_key = 'p7s1'
camera_number = 1
calibraton_file_path_base = "gait3d\\Sequences\\{sequence_key}\\Calibration\\c{camera_number}.xml"
calibraton_file_path_1 = calibraton_file_path_base.format(sequence_key=sequence_key, camera_number=1)
print(calibraton_file_path_1)

calibraton_file_path_2 = calibraton_file_path_base.format(sequence_key=sequence_key, camera_number=2)
print(calibraton_file_path_2)

gait3d\Sequences\p7s1\Calibration\c1.xml
gait3d\Sequences\p7s1\Calibration\c2.xml


In [5]:
with open(calibraton_file_path_1) as file:
    xml_calibration_config = file.read()
    print(xml_calibration_config)

<?xml version="1.0" encoding="UTF-8"?>
<Camera name="c1">
	<Geometry width="960" height="540" ncx="9.600000e+002" nfx="9.600000e+002" dx="1.000000e+000" dy="1.000000e+000" dpx="1.000000e+000" dpy="1.000000e+000"/>
	<Intrinsic focal="5.561470e+002" kappa1="2.720130e-007" cx="4.800000e+002" cy="2.700000e+002" sx="1.001630e+000"/>
	<Extrinsic tx="2.564550e+001" ty="6.130300e+002" tz="3.444080e+003" rx="-3.121731e+000" ry="1.353192e+000" rz="1.582054e+000"/>
</Camera>



In [ ]:
import numpy as np
import cv2
from xml.etree import ElementTree as ET

def parse_camera_params(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    intrinsic = root.find("Intrinsic")
    extrinsic = root.find("Extrinsic")
    
    focal = float(intrinsic.get("focal"))
    cx = float(intrinsic.get("cx"))
    cy = float(intrinsic.get("cy"))
    sx = float(intrinsic.get("sx"))
    
    K = np.array([[focal * sx, 0, cx],
                  [0, focal, cy],
                  [0, 0, 1]])
    
    tx = float(extrinsic.get("tx"))
    ty = float(extrinsic.get("ty"))
    tz = float(extrinsic.get("tz"))
    rx = float(extrinsic.get("rx"))
    ry = float(extrinsic.get("ry"))
    rz = float(extrinsic.get("rz"))
    
    R, _ = cv2.Rodrigues(np.array([rx, ry, rz]))
    t = np.array([[tx], [ty], [tz]])

    return K, R, t

def triangulate_points(K1, R1, t1, K2, R2, t2, points1, points2):
    P1 = K1 @ np.hstack((R1, t1))
    P2 = K2 @ np.hstack((R2, t2))
    
    points1_h = cv2.convertPointsToHomogeneous(points1).reshape(-1, 3).T
    points2_h = cv2.convertPointsToHomogeneous(points2).reshape(-1, 3).T
    print(f"{P1 = } \n {points1_h = }")
    print(f"{P2 = } \n {points2_h = }")
    
    points_4d = cv2.triangulatePoints(P1, P2, points1_h[:2], points2_h[:2])
    print(f"{points_4d= }")
    points_3d = points_4d[:3] / points_4d[3]
    return points_3d.T

K1, R1, t1 = parse_camera_params(calibraton_file_path_1)
K2, R2, t2 = parse_camera_params(calibraton_file_path_2)
points1 = np.array([[1000, 1000], [960, 540], [480, 270]])  # replace with proper mediapipe points
points2 = np.array([[1000, 1000], [960, 540], [480, 270]])  # random poitns

points_3d = triangulate_points(K1, R1, t1, K2, R2, t2, points1, points2)

print("Triangulated 3D points:")
for point in points_3d:
    print(tuple(point))

In [84]:
from utils.mediapipe_estimator import MediaPipeEstimator
from scripts.frame_iterator import video_frame_iterator
from scripts.parsers import parse_sequences
from math import inf
import numpy as np
import cv2
from xml.etree import ElementTree as ET

def parse_camera_params(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    intrinsic = root.find("Intrinsic")
    extrinsic = root.find("Extrinsic")
    
    focal = float(intrinsic.get("focal"))
    cx = float(intrinsic.get("cx"))
    cy = float(intrinsic.get("cy"))
    sx = float(intrinsic.get("sx"))
    
    K = np.array([[focal * sx, 0, cx],
                  [0, focal, cy],
                  [0, 0, 1]], dtype=np.float32)
    
    tx = float(extrinsic.get("tx"))
    ty = float(extrinsic.get("ty"))
    tz = float(extrinsic.get("tz"))
    rx = float(extrinsic.get("rx"))
    ry = float(extrinsic.get("ry"))
    rz = float(extrinsic.get("rz"))
    
    R, _ = cv2.Rodrigues(np.array([rx, ry, rz]))
    t = np.array([[tx], [ty], [tz]], dtype=np.float32)

    return K, R, t


def triangulate_points(K1, R1, t1, K2, R2, t2, points1, points2):
    P1 = K1 @ np.hstack((R1, t1))
    P2 = K2 @ np.hstack((R2, t2))
    
    # points1_h = cv2.convertPointsToHomogeneous(points1)[:, 0, :].T
    # points2_h = cv2.convertPointsToHomogeneous(points2)[:, 0, :].T
    
    points_4d = cv2.triangulatePoints(P1, P2, points1.T, points2.T)
    points_3d = points_4d[:3] / points_4d[3]
    return points_3d.T

def triangulate_frames_from_video(sequence_key: str, camera_numbers: list[int], frames_number=inf, verbose=False):
    file_path = 'gait3d\\ListOfSequences.txt'
    sequence_info = parse_sequences(file_path)[sequence_key]

    if len(camera_numbers) < 2:
        raise Exception("Two cameras are required to triangulate coordinates.") #TODO: check also if c_nums in [1,2,3,4]

    avi_paths = []
    xml_config_paths = []
    
    for camera_number in camera_numbers:
        avi_file_name = f"c{camera_number}_{(4 - len(str(sequence_info['start_frame']))) * '0' + str(sequence_info['start_frame'])}"
        avi_seq_path = f".\\gait3d\\Sequences\\{sequence_key}\\Images\\{avi_file_name}.avi"
        avi_paths.append(avi_seq_path)
        
        calibraton_xml_file_path = f"gait3d\\Sequences\\{sequence_key}\\Calibration\\c{camera_number}.xml" 
        xml_config_paths.append(calibraton_xml_file_path)


    K1, R1, t1 = parse_camera_params(xml_config_paths[0])
    K2, R2, t2 = parse_camera_params(xml_config_paths[1])
    
    estimator_1 = MediaPipeEstimator()
    estimator_2 = MediaPipeEstimator()
    
    video_1_frames = video_frame_iterator(avi_paths[0], min(frames_number, sequence_info['number_of_frames']))
    video_2_frames = video_frame_iterator(avi_paths[1], min(frames_number, sequence_info['number_of_frames']))

    if verbose:
        print(avi_paths)
        print(xml_config_paths)

    video_points_3d = []

    for (i, vid_1_frame), (_, vid_2_frame) in zip(video_1_frames, video_2_frames):
        points_1 = np.array(estimator_1.predict_for_frame(i, vid_1_frame), dtype=np.float32)
        points_2 = np.array(estimator_2.predict_for_frame(i, vid_2_frame), dtype=np.float32)

        if verbose:
            print(f"{points_1 = }")
            print(f"{points_2 = }")

        points_3d = triangulate_points(K1, R1, t1, K2, R2, t2, points_1, points_2)
        video_points_3d.append(points_3d)
        
        if verbose:
            print(points_3d)
        
    return video_points_3d
        

In [85]:
points = triangulate_frames_from_video('p7s1', [1, 2], frames_number=3, verbose=True)

['.\\gait3d\\Sequences\\p7s1\\Images\\c1_0090.avi', '.\\gait3d\\Sequences\\p7s1\\Images\\c2_0090.avi']
['gait3d\\Sequences\\p7s1\\Calibration\\c1.xml', 'gait3d\\Sequences\\p7s1\\Calibration\\c2.xml']
points_1 = array([[811., 130.],
       [813., 125.],
       [814., 124.],
       [815., 124.],
       [812., 125.],
       [813., 124.],
       [813., 124.],
       [824., 122.],
       [821., 123.],
       [817., 135.],
       [815., 135.],
       [858., 154.],
       [822., 153.],
       [861., 201.],
       [816., 198.],
       [852., 244.],
       [810., 231.],
       [852., 256.],
       [807., 242.],
       [845., 255.],
       [807., 242.],
       [844., 251.],
       [808., 240.],
       [840., 236.],
       [818., 233.],
       [825., 297.],
       [812., 292.],
       [818., 352.],
       [806., 346.],
       [822., 363.],
       [811., 357.],
       [790., 358.],
       [780., 350.]], dtype=float32)
points_2 = array([[479., 158.],
       [476., 153.],
       [474., 153.],
      

In [57]:
points[0][:, 0]

array([7748.105 , 7712.964 , 7534.209 , 7515.4585, 7823.407 , 7758.5845,
       7782.843 , 6952.0933, 7367.9297, 7623.3086, 7529.796 , 6417.858 ,
       6172.9697, 4801.3066, 4508.8286, 3436.8313, 3485.7002, 3205.5596,
       3324.6807, 3246.2852, 3324.0503, 3328.8018, 3387.3862, 3488.8242,
       3469.0046, 2342.4878, 2399.067 , 1685.9576, 1784.0144, 1603.8627,
       1703.4572, 1495.7639, 1595.591 ], dtype=float32)

In [58]:
import mediapipe as mp
mp_pose = mp.solutions.pose

landmarks_num = 33
pose_landmark_names = [mp_pose.PoseLandmark(i).name for i in range(landmarks_num)]

print(pose_landmark_names)

['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER', 'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']


In [59]:
import plotly.graph_objects as go
import plotly.io as pio
import mediapipe as mp

pio.renderers.default = 'iframe'

def display_triang_points(points, frame_number=0):
    mp_pose = mp.solutions.pose
    pose_landmark_names = [mp_pose.PoseLandmark(i).name for i in range(landmarks_num)]

    fig = go.Figure(data=[go.Scatter3d(x=points[frame_number][:, 0], 
                                       y=points[frame_number][:, 1], 
                                       z=points[frame_number][:, 2], 
                                       mode='markers', 
                                       marker=dict(size=5, color='blue'),
                                       text=pose_landmark_names)])
    
    fig.update_layout(scene=dict(
                            xaxis_title='X',
                            yaxis_title='Y',
                            zaxis_title='Z',
                            xaxis=dict(range=[-10000, 10000]),
                            yaxis=dict(range=[-6000, 14000]),
                            zaxis=dict(range=[-10000, 10000]),
                            aspectmode='cube', 
                        ),
                        title='3D joints plot from triangulation (mediapipe points)',
                        width=800,
                        height=800,
                        )
    
    fig.show()

display_triang_points(points)

In [61]:
points_2 = triangulate_frames_from_video('p2s1', [2, 3], 1)
display_triang_points(points_2)

In [62]:
points_3 = triangulate_frames_from_video('p2s1', [1, 4], 1)
display_triang_points(points_3)

In [63]:
points_4 = triangulate_frames_from_video('p2s1', [3, 4], 1)
display_triang_points(points_4)

In [64]:
points_test = triangulate_frames_from_video('p4s1', [3, 2], 1)
display_triang_points(points_test)

In [70]:
points_test = triangulate_frames_from_video('p7s1', [1, 2])
display_triang_points(points_test, 0)

In [71]:
display_triang_points(points_test, 40)

In [72]:
display_triang_points(points_test, 80)

In [76]:
points_test = triangulate_frames_from_video('p7s1', [2, 3], 1)
display_triang_points(points_test, 0)

In [82]:
points_test = triangulate_frames_from_video('p3s1', [3, 2], 61)
display_triang_points(points_test, 0)

In [83]:
display_triang_points(points_test, 60)